In [1]:
import re

def embeddings_path(dataset_name, embedding_size, bpe=False):
    if bpe:
        return f"embeddings/{dataset_name}_embeddings_{embedding_size}_bpe.txt"
    return f"embeddings/{dataset_name}_embeddings_{embedding_size}.txt"

def tokenize_text(input_file, output_file, tokenizer, regex, new_line_marker='\n', max_sentences=None):
    sentences = []
    with open(input_file, 'r', encoding='utf-8') as fin, \
         open(output_file, 'w', encoding='utf-8') as fout:
        pattern = re.compile(regex, re.UNICODE)
        for line in fin:
            tokens = pattern.findall(line.lower())
            if not tokens:
                continue
            if tokenizer == 'word':
                sentences.append(tokens)
                sentences[-1].append(new_line_marker)
            elif tokenizer == 'char':
                words = []
                for token in tokens:
                    token_chars = list(token)
                    token_chars.append('</w>')
                    tokenized_word = ' '.join(token_chars)
                    words.append(tokenized_word)
                sentences.append(words)
                sentences[-1].append(new_line_marker)
            fout.write(" ".join(sentences[-1]) + ' ')
            if max_sentences is not None and len(sentences) >= max_sentences:
                break
    return sentences

def load_sentences(file_path):
    sentences = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            tokens = line.strip().split()
            sentences.append(tokens)
    return sentences


In [3]:
dataset_path = "datasets/harrypotter.txt"
# dataset_path = "datasets/chan_dialogues.txt"
# dataset_path = "datasets/war.txt"
dataset_name = dataset_path.split('/')[-1].split('.')[0]

embeddings_d = [100, 500, 1000]
max_vocab_size = 120000

max_sentences = None

bpe = True

# pattern = r'\w+|[^\w\s]' # со знаками препинания
pattern =  r"\w+"        # только слова   
tokenizer = 'char' if bpe else 'word'
tokenize_text(dataset_path, f"sentences/{dataset_name}_sentences_{tokenizer}.txt", tokenizer=tokenizer, regex=pattern, new_line_marker='</n>', max_sentences=max_sentences)
sentences = load_sentences(f"sentences/{dataset_name}_sentences_{tokenizer}.txt")

total_words = sum(len(sentence) for sentence in sentences)
print(f"Total number of words: {total_words}")
print(f"Total number of sentences: {len(sentences)}")

Total number of words: 6000812
Total number of sentences: 1


In [6]:
import subprocess

if bpe:
    subprocess.run(["./bpe_tokenizer",
                    f"sentences/{dataset_name}_sentences_{tokenizer}.txt",
                    f"sentences/{dataset_name}_sentences_bpe.txt",
                   '3000'
                   ], check=True)

Прочитано 1 строк. (0.04712 сек)
Итерация 1: найдено 5141 уникальных пар. (1.47653 сек)
Лучшая пара: о + </w> -> о</w>
Слияние 1 завершено. (1.84997 сек)
Итерация 101: найдено 21519 уникальных пар. (89.0652 сек)
Лучшая пара: б + ы -> бы
Слияние 101 завершено. (89.2367 сек)
Итерация 201: найдено 44529 уникальных пар. (167.161 сек)
Лучшая пара: ако + й</w> -> акой</w>
Слияние 201 завершено. (167.277 сек)
Итерация 301: найдено 72396 уникальных пар. (252.666 сек)
Лучшая пара: г + е -> ге
Слияние 301 завершено. (252.779 сек)
Итерация 401: найдено 102418 уникальных пар. (328.059 сек)
Лучшая пара: хо + ди -> ходи
Слияние 401 завершено. (328.186 сек)
Итерация 501: найдено 132135 уникальных пар. (417.125 сек)
Лучшая пара: ю + сь</w> -> юсь</w>
Слияние 501 завершено. (417.246 сек)
Итерация 601: найдено 161600 уникальных пар. (525.338 сек)
Лучшая пара: ве + н -> вен
Слияние 601 завершено. (525.435 сек)
Итерация 701: найдено 188559 уникальных пар. (612.089 сек)
Лучшая пара: пос + лед -> послед
Сли

In [5]:
if bpe:
    sentences = load_sentences(f"sentences/{dataset_name}_sentences_bpe.txt")
    total_words = sum(len(sentence) for sentence in sentences)
    print(f"Total number of words after BPE: {total_words}")

Total number of words after BPE: 1421051


In [11]:
import subprocess

for d in embeddings_d:
    window_size = 5
    num_negative_examples = 5
    num_epoch = 5 + d // 500
    learning_rate = 0.025
    subprocess.run(["./generate_embeddings",            # executable
                    f"sentences/{dataset_name}_sentences_{'bpe' if bpe else 'word'}.txt",       # path to tokenized sentences
                    embeddings_path(dataset_name, d, bpe=bpe),   # path to save embeddings to
                    str(d),                             # word vec dim
                    str(max_vocab_size),                # max size of vocabulary
                    str(window_size),                                # windows size
                    str(num_negative_examples),                                # num negative examples
                    str(num_epoch),                                # num epoch
                    str(learning_rate),                             # learning rate
                    ], check=True)

Текст: sentences/harrypotter_sentences_bpe.txt
Эмбеддинги: embeddings/harrypotter_embeddings_100_bpe.txt
Размер эмбеддингов (d): 100
Макс. размер словаря: 120000
Размер окна: 5
Количество отрицательных примеров: 5
Количество эпох: 5
Скорость обучения (lr): 0.025

Загрузка текста и создание словаря...
Текст прочитан. (0.127667 сек)
Количество строк: 1
Количество уникальных слов: 3011

Словарь создан. (0.128381 сек)
Размер словаря: 3011
Макс. размер словаря: 120000
Примеры словаря: 
the</w> (52269) -> 0; </n> (38195) -> 1; and</w> (28090) -> 2; to</w> (27261) -> 3; he</w> (22343) -> 4; a</w> (22282) -> 5; of</w> (22076) -> 6; harry</w> (18367) -> 7; s</w> (16007) -> 8; was</w> (15691) -> 9; 

Предложения закодированы. (0.172589 сек)
Количество закодированных предложений: 1
Примеры закодированных предложений: 
65 1656 2 269 2403 6 1069 748 2425 1846 


In [6]:
import numpy as np

def load_embeddings(file_path):
    with open(file_path, encoding='utf-8') as f:
        header = f.readline().strip().split()
        vocab_size = int(header[0])
        d = int(header[1])
        embeddings = np.zeros((vocab_size, d), dtype=np.float32)
        word2index = {}
        index2word = []
        for i, line in enumerate(f):
            parts = line.strip().split()
            if len(parts) != d + 1:
                continue 
            word = parts[0]
            vector = np.array(parts[1:], dtype=np.float32)
            embeddings[i] = vector
            word2index[word] = i
            index2word.append(word)
    return embeddings, word2index, index2word

# embeddings_file = embeddings_path(dataset_name, 100)  
# embeddings, word2index, index2word = load_embeddings(embeddings_file)
# print("Загруженная матрица эмбеддингов:", embeddings.shape)
# print("Примеры слов:", list(word2index.keys())[:10])

# print("Примеры векторов:", list(embeddings)[:3])


In [8]:
def generate_training_examples_from_sentence(sentence, L):
    """
    Генерирует обучающие примеры (контекст, целевой токен) из предложения.
    где контекст — это L последовательных токенов, а целевой токен — следующий за ними.
    """
    if len(sentence) < L + 1:
        return  # Недостаточно токенов для формирования хотя бы одного примера.
    for i in range(len(sentence) - L):
        context = sentence[i : i + L]
        target = sentence[i + L]
        yield context, target
        
def batch_generator_pretokenized(tokenized_sentences, word2index, L, batch_size=512):
    """
    Генератор батчей обучающих примеров из токенизированных предложений
    """
    while True:
        X, y = [], []
        for sentence in tokenized_sentences:
            for context, target in generate_training_examples_from_sentence(sentence, L):
                X.append([word2index.get(w, 0) for w in context])
                y.append(word2index.get(target, 0))
                if len(X) >= batch_size:
                    yield np.array(X), np.array(y)
                    X, y = [], []  # Очищаем батч
        if X:  # Возвращаем остаток батча
            yield np.array(X), np.array(y)

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.backend import clear_session as clear_keras_session
from tensorflow.keras.callbacks import EarlyStopping

L = 5    # Длина контекста для предсказания
lstm_units = 128
dh = 512  # Размер скрытого слоя
epochs = 10

batch_size = 512

steps_per_epoch = np.ceil(total_words / batch_size).astype(int)

for d in embeddings_d:
    embeddings_file = embeddings_path(dataset_name, d, bpe=bpe)
    embeddings, word2index, index2word = load_embeddings(embeddings_file)

    vocab_size = len(word2index)

    print(f"Размер словаря: {vocab_size}, размер эмбеддинга: {d}")
    print(f"Всего слов: {total_words}, шагов на эпоху: {steps_per_epoch}")

    clear_keras_session()  # Очистка сессии Keras

    # Создаем модель
    
#     model = models.Sequential(name="word_prediction_model")
#     model.add(layers.Embedding(input_dim=vocab_size, 
#                                 output_dim=d, 
#                                 weights=[embeddings], 
#                                 trainable=False, 
#                                 input_length=L, 
#                                 name="pretrained_embedding"))
#     model.add(layers.LSTM(lstm_units, name="lstm_layer"))
#     model.add(layers.Dense(dh, activation='relu', name="dense_hidden"))
#     model.add(layers.Dense(vocab_size, activation='softmax', name="output_softmax"))

    inputs = layers.Input(shape=(L,), name="context_input")
    embedding_layer = layers.Embedding(input_dim=vocab_size, 
                                       output_dim=d, 
                                       weights=[embeddings], 
                                       input_length=L, 
                                       trainable=False, 
                                       name="pretrained_embedding")(inputs)
    lstm_output = layers.LSTM(lstm_units, name="lstm_layer")(embedding_layer)
    hidden = layers.Dense(dh, activation='relu', name="dense_hidden")(lstm_output)
    outputs = layers.Dense(vocab_size, activation='softmax', name="output_softmax")(hidden)
    
    model = models.Model(inputs=inputs, outputs=outputs, name="word_prediction_model")
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    
    # Обучаем модель предсказания
    model.fit(batch_generator_pretokenized(sentences, word2index, L, batch_size), 
            epochs=epochs, 
            steps_per_epoch=steps_per_epoch,
            callbacks=[EarlyStopping(monitor='loss', patience=3)])

    # Сохраняем модель
    model.save(f"models/{dataset_name}_model_{d}_{'bpe' if bpe else 'word'}.keras")

2025-03-22 19:21:42.972363: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742660502.981586  646340 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742660502.984007  646340 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742660502.992047  646340 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742660502.992059  646340 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742660502.992061  646340 computation_placer.cc:177] computation placer alr

Размер словаря: 3011, размер эмбеддинга: 100
Всего слов: 1421051, шагов на эпоху: 2776


/home/rennelv/etu/ai/.conda/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1742660507.119130  646340 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2152 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "word_prediction_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ pretrained_embedding            │ ?                      │       301,100 │
│ (Embedding)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer (LSTM)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_hidden (Dense)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_softmax (Dense)          │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ pretrained_embedding            │ ?                      │     3,011,000 │
│ (Embedding)                     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer (LSTM)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_hidden (Dense)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_softmax (Dense)          │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,011,000 (11.49 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,011,000 (11.49 MB)

Epoch 1/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 30s 10ms/step - accuracy: 0.0794 - loss: 5.9705
Epoch 2/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.1761 - loss: 4.7491
Epoch 3/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2064 - loss: 4.4363
Epoch 4/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2225 - loss: 4.2746
Epoch 5/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2326 - loss: 4.1667
Epoch 6/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - accuracy: 0.2404 - loss: 4.0862
Epoch 7/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2462 - loss: 4.0231
Epoch 8/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2511 - loss: 3.9714
Epoch 9/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 28s 10ms/step - accuracy: 0.2554 - loss: 3.9278
Epoch 10/10
2776/2776 ━━━━━━━━━━━━━━━━━━━━ 29s 10ms/step - accuracy: 0.2592 - loss: 3.8905


In [10]:
import numpy as np
from tensorflow.keras import models
from tensorflow.keras.backend import clear_session as clear_keras_session
import random

def evaluate_perplexity(model, test_sentence, L, word2index):
    """
    Для каждого окна длины L модель предсказывает вероятность истинного следующего слова
    perplexity = exp( -(1/N) * sum(log(P(w_i|context)))
    где N - количество предсказаний, а P(w_i|context) - вероятность истинного слова
    """
    total_log_prob = 0.0
    count = 0

    test_tokens = test_sentence.lower().split()
    test_indices = [word2index.get(word, 0) for word in test_tokens]
        
    # Если тестовая последовательность слишком короткая возвращаем бесконечность
    if len(test_indices) <= L:
        return float('inf')

    # Проходим по тестовой последовательности, формируя окна длины L
    for i in range(len(test_indices) - L):
        context = test_indices[i:i+L]         # Контекстное окно из L токенов
        true_word = test_indices[i+L]           # Истинное следующее слово
        
        # Получаем распределение вероятностей для следующего слова
        pred = model.predict(np.array([context]), verbose=0)[0]
        # Берем вероятность истинного слова; добавляем маленькую константу для стабильности log
        prob = pred[true_word] + 1e-10
        
        # Суммируем логарифмы вероятностей
        total_log_prob += np.log(prob)
        count += 1

    if count == 0:
        return float('inf')
        
    # Вычисляем среднее логарифмическое значение
    avg_log_prob = total_log_prob / count
    # Перплексия – экспонента от отрицательной средней логарифмической вероятности
    perplexity = np.exp(-avg_log_prob)
    return perplexity

def tokenize_bpe(sentence, word2index):
    """
    Токенизирует предложение на токены, используя словарь word2index.
    Перебирает все возможные срезы для корректного разбиения.
    """
    tokens = []
    for word in sentence.lower().split():
        word = word + '</w>'  # Добавляем маркер конца слова
        start = 0
        while start < len(word):
            for end in range(len(word), start, -1):  # Перебираем все возможные срезы
                subword = word[start:end]
                if subword in word2index:
                    tokens.append(subword)
                    start = end - 1  # Сдвигаем начало на конец найденного токена
                    break
            else:
                # Если ни один токен не найден, добавляем символ как отдельный токен
                tokens.append(word[start])
            start += 1
    return tokens

# Длина контекстного окна для предсказания следующего слова
L = 5  
# Длина тестового предложения
S = 15
# Формируем один длинный список из всех слов текста
sentences_unpacked = []
for sentence in load_sentences(f"sentences/{dataset_name}_sentences_{'bpe' if bpe else 'word'}.txt"):
    sentences_unpacked.extend(sentence)

embeddings_file = embeddings_path(dataset_name, 100, bpe=bpe)
_, word2index, index2word = load_embeddings(embeddings_file)

# test_sentence = 'my name is harry'
# bpe_tokens = tokenize_bpe(test_sentence, word2index)

# print(f"Токены BPE: {bpe_tokens}")

# Выбираем случайное окно в общем списке
i = random.randint(0, len(sentences_unpacked) - S - 1)
# Берем окно из S+1 слов
test_sentence = " ".join(sentences_unpacked[i:i+S+1])

# Преобразуем предложение в список токенов (слова приводятся к нижнему регистру)
test_tokens = test_sentence.lower().split()

# Если число токенов меньше L, дополняем их пробелами; если больше, берем последние L токенов.
if len(test_tokens) < L:
    test_tokens = [' '] * (L - len(test_tokens)) + test_tokens
elif len(test_tokens) > L:
    test_tokens = test_tokens[-L:]

print(f"Тестовое предложение: '{test_sentence.replace('</w>', ' ')}'")

# Преобразуем токены в индексы
test_indices = [word2index.get(word, 0) for word in test_tokens]
print(f"Тестовые индексы: {test_indices}")

# Для каждого размера эмбеддингов оцениваем модель
for d in embeddings_d:
    # Загружаем модель, соответствующую текущей размерности эмбеддингов
    clear_keras_session()  # Очистка сессии Keras
    model = models.load_model(f"models/{dataset_name}_model_{d}_{'bpe' if bpe else 'word'}.keras")

    # Предсказываем следующее слово по текущему контексту
    predicted = model.predict(np.array([test_indices]))
    predicted_index = np.argmax(predicted)
    predicted_word = index2word[predicted_index]
    print(f"[{d}] Предсказанное следующее слово для '{''.join([index2word[i] for i in test_indices]).replace('</w>', ' ')}': {predicted_word}")
    
    # Вычисляем перплексию для тестового предложения
    perplexity = evaluate_perplexity(model, test_sentence, L, word2index)
    print(f"[{d}] Перплексия на тестовом предложении: {perplexity:.2f}")

    generated_words = []
    current_sequence = test_indices
    for _ in range(100):
        predicted = model.predict(np.array([current_sequence]), verbose=0)
        predicted_index = np.argmax(predicted)
        generated_words.append(index2word[predicted_index])
        current_sequence = current_sequence[1:] + [predicted_index]
    
    if bpe:
        s = ''.join(generated_words).split('</w>')
        print(f"Сгенерированные слова: ", *s)
    else:
        print(f"Сгенерированные слова: {' '.join(generated_words)}")


Тестовое предложение: 'the  only  way  to  sort  this  out  </n> i  m  not  running  around  after  him  trying '
Тестовые индексы: [1120, 87, 187, 22, 329]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
[100] Предсказанное следующее слово для 'running around after him trying ': to</w>
[100] Перплексия на тестовом предложении: 60.64
Сгенерированные слова:  to fight the sword of gryffindor s sword and the elder wand and the snake s eyes were fixed upon the ground and the snake s head was still holding the wand of the death eaters were now standing in the middle of the forest and the snake s eyes were fixed upon the ground and the snake s head was still holding the wand of the death eaters were now standing in the middle of the forest and the snake s eyes were fixed upon the ground and the snake s head was still holding the wand of 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
[500] Предсказанное следующее слово для 'running around after him trying ': to</w>
[500] Перплексия на тестовом предложении: 7